In [6]:
import json
import os
import copy

from mbas.experiment_planning.model_configurator import MBASTrainerConfigurator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
new_config_fpath = "/home/bryan/data/mbas_nnUNet_preprocessed/Dataset101_MBAS/plans_2024_09_11.json"

In [11]:
configurator = MBASTrainerConfigurator(
    os.path.basename(os.path.splitext(new_config_fpath)[0]),
    dataset_name = "Dataset101_MBAS",
)

In [12]:
configurator.configurations["ResEncUNet_p20_256_dil2_bd_cascade_ResEncUNet_08_27_nopost"] = configurator.set_params(
    patch_size=(20,256,256),
    data_identifier = "nnUNetPlans_3d_fullres",
    spacing = (2.5, 0.625, 0.625),
    probabilistic_oversampling = True,
    oversample_foreground_percent = 1.0,
    sample_class_probabilities = {1: 0.5, 2: 0.25, 3: 0.25},
    batch_dice = True,
    voxel_sample_z_coverage = False,
).set_cascade(
    cascaded_mask_dilation=2,
    is_cascaded_mask=True,
    previous_stage="Dataset104_ResEncUNet_3d_lowres_for25_drop50_slim96",
).nnUNetResEncUNet(
    features_per_stage= [32,64,128,256,320,320,320],
    kernel_sizes=[
        [1, 3, 3],
        [1, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
    ],
    strides=[
        [1, 1, 1],
        [1, 2, 2],
        [1, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        [1, 2, 2],
        [1, 2, 2],
    ],
    n_blocks_per_stage=[1, 3, 4, 6, 6, 6, 6],
    n_conv_per_stage_decoder=[1, 1, 1, 1, 1, 1],   
)

In [13]:
configurator.save(new_config_fpath)